In [1]:
from indra.sources import minerva
from indra.sources.minerva import minerva_client
from indra.sources.minerva.id_mapping import indra_db_refs_from_minerva_refs
config = minerva_client.get_config(minerva_client.default_map_name)
project_id = minerva_client.get_project_id_from_config(config)
minerva_client.get_models(project_id, minerva_client.default_map_name)
models = minerva_client.get_models(project_id)
all_model_elements = minerva_client.get_all_model_elements(
    models, project_id, minerva_client.default_map_name)
phenotypes = [element for element in all_model_elements if element.get('type') == 'Phenotype']
phenotypes = sorted(phenotypes, key=lambda x: x['name'].lower())
phenotype_refs = [indra_db_refs_from_minerva_refs(minerva_client.get_element_references(p))
                  for p in phenotypes]

INFO: [2021-04-15 10:22:45] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /Users/ben/.indra/bio_ontology/1.12/bio_ontology.pkl


In [2]:
from indra.ontology.bio import bio_ontology
from indra.statements.agent import get_grounding
records = [{'name': ref['TEXT'], 'grounding': '%s:%s' % get_grounding(ref),
            'standard name': str(bio_ontology.get_name(*get_grounding(ref)))}
           for ref in phenotype_refs]
import pandas
pandas.set_option('display.max_rows', None)
df = pandas.DataFrame.from_records(records)
df.replace('None:None', '', inplace=True)
df.replace('None', '', inplace=True)

In [21]:
df

,name,grounding,standard name
0,"Diabetes mellitus, type II",MESH:D003924,"Diabetes Mellitus, Type 2"
1,ABO blood group system,MESH:D000017,ABO Blood-Group System
2,accumulation of misfolded protein in ER,GO:GO:0006986,response to unfolded protein
3,Activity of sodium channels,,
4,aging,GO:GO:0007568,aging
5,Apoptosis,GO:GO:0006915,apoptotic process
6,Apoptosis,GO:GO:0006915,apoptotic process
7,Apoptosis,GO:GO:0006921,cellular component disassembly involved in exe...
8,Apoptosis,GO:GO:0006915,apoptotic process
9,apoptosis,GO:GO:0006915,apoptotic process


In [3]:
df.to_csv('c19dm_phenotypes.csv', index=False)

In [4]:
readouts = [
'cytokine storm',
'ER Stress',
'fibrosis',
'Hypokalemia',
'Immune Response',
'Inflammation',
'inflammatory response',
'neurodegeneration',
'Persistant ER Stress',
'pulmonary edema',
'pulmonary fibrosis',
'SARS-CoV-2 infection',
'SARS-CoV-2 viral entry',
'thrombosis',
'Thrombosis',
'vascular inflammation',
'vascular remodeling',
'viral entry into host cell',
'Viral replication',
'viral replication cycle',
'viral RNA genome replication',
'virus replication',
'Virus replication cycle',
'virus_replication',
]

In [5]:
import copy
from indra.sources import minerva
mp = minerva.process_from_web()
dm_stmts = mp.statements
from indra.statements import Activation, Agent, Evidence
def find_agent(readout):
    for stmt in dm_stmts:
        for agent in stmt.real_agent_list():
            if agent.db_refs.get('TEXT') == readout:
                return copy.deepcopy(agent)

exclude_list = ['Microvessels']
            
readout_stmts = []
target_agent = Agent('COVID-19 adverse outcomes')
for readout in readouts:
    agent = find_agent(readout)
    if not agent:
        continue
    if agent.name in exclude_list:
        continue
    stmt = Activation(agent, target_agent,
                      evidence=[Evidence(source_api='assertion')])
    readout_stmts.append(stmt)

INFO: [2021-04-15 10:23:18] indra.sources.minerva.processor - Processing model 906
INFO: [2021-04-15 10:23:19] indra.sources.minerva.processor - Got 40 statements from model 906
INFO: [2021-04-15 10:23:19] indra.sources.minerva.processor - Processing model 913
INFO: [2021-04-15 10:23:20] indra.sources.minerva.processor - Got 195 statements from model 913
INFO: [2021-04-15 10:23:20] indra.sources.minerva.processor - Processing model 908
INFO: [2021-04-15 10:23:21] indra.sources.minerva.processor - Got 266 statements from model 908
INFO: [2021-04-15 10:23:21] indra.sources.minerva.processor - Processing model 905
INFO: [2021-04-15 10:23:23] indra.sources.minerva.processor - Got 154 statements from model 905
INFO: [2021-04-15 10:23:23] indra.sources.minerva.processor - Processing model 907
INFO: [2021-04-15 10:23:24] indra.sources.minerva.processor - Got 228 statements from model 907
INFO: [2021-04-15 10:23:24] indra.sources.minerva.processor - Processing model 909
INFO: [2021-04-15 10:23

In [6]:
readout_stmts

[Activation(cytokine storm(), COVID-19 adverse outcomes()),
 Activation(response to endoplasmic reticulum stress(), COVID-19 adverse outcomes()),
 Activation(Fibrosis(), COVID-19 adverse outcomes()),
 Activation(Hypokalemia(), COVID-19 adverse outcomes()),
 Activation(inflammatory response(), COVID-19 adverse outcomes()),
 Activation(Nerve Degeneration(), COVID-19 adverse outcomes()),
 Activation(response to endoplasmic reticulum stress(), COVID-19 adverse outcomes()),
 Activation(Pulmonary Edema(), COVID-19 adverse outcomes()),
 Activation(Pulmonary Fibrosis(), COVID-19 adverse outcomes()),
 Activation(RNA Virus Infections(), COVID-19 adverse outcomes()),
 Activation(RNA Virus Infections(), COVID-19 adverse outcomes()),
 Activation(Thrombosis(), COVID-19 adverse outcomes()),
 Activation(vascular inflammation(), COVID-19 adverse outcomes()),
 Activation(blood vessel remodeling(), COVID-19 adverse outcomes()),
 Activation(viral entry into host cell(), COVID-19 adverse outcomes()),
 Acti

In [7]:
with open('c19dm_readout_stmts.pkl', 'wb') as fh:
    pickle.dump(readout_stmts, fh)